# Linear Regression: Iowa Liquor Sales

### The Data & The Problem

The state of Iowa publishes sales data from its liquor stores (stores with class E licenses).

We can look back at the transactions and search for interesting turns in the data; but how can we use this data set to help us make decisions about the future? 

Using linear regression and machine learning, we can use the data from 2015 and the first quarter of 2016 to predict how stores or geographical areas (zip codes, counties, cities) will perform over the rest of the year.

#### Dependencies

In [20]:
# Libraries

% matplotlib inline

# time
import datetime

# math & stats
import numpy as np
import scipy as sp

# data management
import pandas as pd

# visualizations
import seaborn as sns
import matplotlib.pyplot as plt

# linear regression, r2 scoring metric, metrics in general
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import metrics

# cross val, cross val scoring
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold



#### Import Data

Import the data from CSV, around XXXX rows. 

Use pandas' infer_datetime_format to get dates automatically.

In [6]:
df = pd.read_csv('Iowa_Liquor_Sales_reduced.csv', infer_datetime_format=True)

#### Clean the Data

In its raw form, the data comes in mis-formatted and with non-optimal data-types. We need to remove the "$" characters from the Bottle Cost, Retail, and Sales columns, and then adjust the data type of data and numeric columns

In [7]:
df['State Bottle Cost'] = [dollars.strip('$') for dollars in df['State Bottle Cost']]
df['State Bottle Retail'] = [dollars.strip('$') for dollars in df['State Bottle Retail']]
df['Sale (Dollars)'] = [dollars.strip('$') for dollars in df['Sale (Dollars)']]

df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True)
df['State Bottle Cost'] = pd.to_numeric(df['State Bottle Cost'], errors='coerce')
df['State Bottle Retail'] = pd.to_numeric(df['State Bottle Retail'], errors='coerce')
df['Sale (Dollars)'] = pd.to_numeric(df['Sale (Dollars)'], errors='coerce')
df['Zip Code'] = pd.to_numeric(df['Zip Code'], errors='coerce')
df['County Number'] = pd.to_numeric(df['County Number'], errors='coerce')
df['Store Number'] = pd.to_numeric(df['Store Number'], errors='coerce')

##### More cleaning

There are some divide by zero issues and lines where the price per litre approaches infinity. Finding where this occurs and replacing the errors with the correct value solves the problem

In [8]:
df.sort_values(by=["Bottle Volume (ml)"], inplace=True)

df.loc[df['Bottle Volume (ml)'] == 0]

df.set_value([2007100,815951],'Bottle Volume (ml)',50)


,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
2007100,2015-05-06,2614,DAVENPORT,52807.0,82.0,Scott,1031200.0,VODKA FLAVORED,259,941063,Burnett's Pink Lemonade Vodka Mini,50,4.25,6.38,12,76.56,0.0,0.00
815951,2015-11-18,2614,DAVENPORT,52807.0,82.0,Scott,1031200.0,VODKA FLAVORED,259,941063,Burnett's Pink Lemonade Vodka Mini,50,4.25,6.38,12,76.56,0.0,0.00
2280519,2015-03-18,2642,PELLA,50219.0,63.0,Marion,1081380.0,MISCELLANEOUS SCHNAPPS,55,984160,99 Apples Mini Display,50,3.30,4.95,24,118.80,1.2,0.32
593084,2015-12-22,4669,DES MOINES,50312.0,77.0,Polk,1012200.0,SCOTCH WHISKIES,885,901307,Blair Athol,50,5.83,8.75,6,52.50,0.3,0.08
2004339,2015-05-06,2614,DAVENPORT,52807.0,82.0,Scott,1031200.0,VODKA FLAVORED,259,941550,Burnett's Peach Vodka Miniature,50,4.25,6.38,12,76.56,0.6,0.16
907472,2015-11-04,4669,DES MOINES,50312.0,77.0,Polk,1081900.0,MISC. AMERICAN CORDIALS & LIQUEURS,885,901327,Yahara Bay Orangecello,50,4.30,6.45,6,38.70,0.3,0.08
2570340,2015-01-28,4669,DES MOINES,50312.0,77.0,Polk,1052010.0,IMPORTED GRAPE BRANDIES,885,900418,Cognac Napoleon XO,50,8.52,12.78,6,76.68,0.3,0.08
2130678,2015-04-15,4669,DES MOINES,50312.0,77.0,Polk,1012200.0,SCOTCH WHISKIES,885,987876,Braeval,50,6.47,9.71,6,58.26,0.3,0.08
593076,2015-12-22,4669,DES MOINES,50312.0,77.0,Polk,1011200.0,STRAIGHT BOURBON WHISKIES,885,901190,Yahara Bay American Whiskey,50,4.60,6.90,6,41.40,0.3,0.08
1637990,2015-07-06,5102,MOUNT VERNON,52314.0,57.0,Linn,1062300.0,FLAVORED RUM,35,943131,Bacardi Limon,50,5.47,8.21,12,98.52,0.6,0.16


#### Intermediate Calculations

The metrics we have are OK, but we could use some more advanced metrics that tell us a little more about the transactions: 
* Margin, the difference between the cost of the items sold and their retail value 
* Price per Liter, which tells us how relatively 'high end' the items sold are; higher-end items costing more per liter.

In [9]:
df['Margin'] = df['State Bottle Retail'] - df['State Bottle Cost']
df['Price per Liter'] = df['State Bottle Retail']/(df['Bottle Volume (ml)']/1000)

#### Slicing and Masking

In order to figure out our predicted 2016 values, we need to build a model using our 2015 Q1 data to predict our 2015 year totals, and then applying that model to our 2016 Q1 values to predict what our 2016 year end values should be.

In order to do this, we need 3 different slices of our data:
* 2015 Q1
* 2015 Year End
* 2016 Q1

We'll get this by calling a mask using a start date and end date on our dataframe, then grouping the data by store and applying some aggregate functions on our data to produce sliced dataframes with the variables we want to use for modeling.

In [10]:
# 2015 Values

df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales_2015 = df[mask]


# Group by store name
sales_2015 = sales_2015.groupby(by=["Store Number"], as_index=False)

# Compute sums, means
sales_2015 = sales_2015.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0],
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})

# Collapse the column indices
sales_2015.columns = [' '.join(col).strip() for col in sales_2015.columns.values]
# Rename columns

sales_2015.columns = ['store_number','city','2015_sales_sum','2015_sales_mean','county_number','2015_price_per_liter_mean'
                     ,'zip_code','2015_volume_sold_liters_sum','2015_volume_sold_liters_mean','2015_margin_mean']



In [11]:
# 2015 Q1 by store Values

df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20150331")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales_Q1_2015 = df[mask]


# Group by store name
sales_Q1_2015 = sales_Q1_2015.groupby(by=["Store Number"], as_index=False)

# Compute sums, means
sales_Q1_2015 = sales_Q1_2015.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0],
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})

# Collapse the column indices
sales_Q1_2015.columns = [' '.join(col).strip() for col in sales_Q1_2015.columns.values]

# Rename columns
sales_Q1_2015.columns = ['store_number','city','2015_q1_sales_sum','2015_q1_sales_mean','county_number','2015_q1_price_per_liter_mean'
                     ,'zip_code','2015_q1_volume_sold_liters_sum','2015_q1_volume_sold_liters_mean','2015_q1_margin_mean']


In [12]:
# 2016 Q1 by store Values

df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20160101")
end_date = pd.Timestamp("20160331")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales_Q1_2016 = df[mask]


# Group by store name
sales_Q1_2016 = sales_Q1_2016.groupby(by=["Store Number"], as_index=False)

# Compute sums, means
sales_Q1_2016 = sales_Q1_2016.agg({"Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,
                   "Price per Liter": np.mean,
                   "Zip Code": lambda x: x.iloc[0],
                   "City": lambda x: x.iloc[0],
                   "County Number": lambda x: x.iloc[0]})

# Collapse the column indices
sales_Q1_2016.columns = [' '.join(col).strip() for col in sales_Q1_2016.columns.values]

# Rename columns
sales_Q1_2016.columns = ['store_number','city','2016_q1_sales_sum','2016_q1_sales_mean','county_number','2016_q1_price_per_liter_mean'
                     ,'zip_code','2016_q1_volume_sold_liters_sum','2016_q1_volume_sold_liters_mean','2016_q1_margin_mean']


#### Combining our 2015 Q1 and 2015 Year End data into one dataframe

In order to build a model I had to match up my Q1 and Year total data for 2015 into one dataframe, so I can use my Q1 data as my features (X) and my 2015 totals as my target (y)



In [13]:
combined_2015 = pd.merge(sales_Q1_2015, sales_2015, on=['store_number','city','county_number'])

In [14]:
sales_2015_mod = combined_2015[['store_number','city','2015_q1_sales_sum','2015_q1_price_per_liter_mean'
                               ,'zip_code_x','2015_q1_volume_sold_liters_mean','2015_q1_margin_mean','2015_sales_sum']]

### Linear Regression, KFolds Cross Validation

We have our Q1 2015, Year Total 2015, and Q1 2016 data. We can use a linear regression model to train our model with the Q1 2015 data for each store as our features and the 2015 Year-End Sales for each store as our target.

Then, we'll use our model to predict the 2016 Year-End sales using the Q1 2016 data.

#### Set up our features (X) and targets (y)

In [28]:
X = sales_2015_mod[['2015_q1_sales_sum','2015_q1_price_per_liter_mean','2015_q1_margin_mean']]
y = sales_2015_mod['2015_sales_sum']

X_2016 = sales_Q1_2016[['2016_q1_sales_sum','2016_q1_price_per_liter_mean','2016_q1_margin_mean']]


#### KFolds cross validation

In [18]:
kf = KFold(len(y), n_folds=12 ,shuffle=True, random_state=0)


acc_scores = []
fold = 0

for training, testing in kf:
    fold += 1
    print "Fold Start", fold
    
    X_train = X.loc[training]
    X_test = X.loc[testing]
    y_train = y.loc[training]
    y_test = y.loc[testing]


    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    r2score = metrics.r2_score(y_test, y_pred)
    acc_scores.append(r2score)
    print "Fold r2:", r2score
    print "Fold Complete", fold



Fold Start 1
Fold r2: 0.935642481334
Fold Complete 1
Fold Start 2
Fold r2: 0.971985224947
Fold Complete 2
Fold Start 3
Fold r2: 0.981612539963
Fold Complete 3
Fold Start 4
Fold r2: 0.937180255717
Fold Complete 4
Fold Start 5
Fold r2: 0.967441064185
Fold Complete 5
Fold Start 6
Fold r2: 0.96494722462
Fold Complete 6
Fold Start 7
Fold r2: 0.989130273563
Fold Complete 7
Fold Start 8
Fold r2: 0.982457302823
Fold Complete 8
Fold Start 9
Fold r2: 0.969709495013
Fold Complete 9
Fold Start 10
Fold r2: 0.993311112368
Fold Complete 10
Fold Start 11
Fold r2: 0.977672887289
Fold Complete 11
Fold Start 12
Fold r2: 0.982913360659
Fold Complete 12


/Users/johntate/anaconda/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


### Cross Val Scores / Predict

Evaluate using a simpler method of cross validation

In [21]:
model = linear_model.LinearRegression()
cvscores = cross_val_score(model, X, y, cv=12)

cvpred = cross_val_predict(model, X, y, cv=12)
cvr2score = metrics.r2_score(y, cvpred)

#### Compare methods

In [24]:
print "Mean r2 from KFolds: ", np.mean(acc_scores),"\n"
print "Cross-validated scores:", cvscores, "\n"
print "Cross-Predicted Accuracy:", cvr2score

Mean r2 from KFolds:  0.971166935207 

Cross-validated scores: [ 0.96458402  0.98127487  0.95324412  0.9432093   0.96340519  0.9477438
  0.95490295  0.9622457   0.84254856  0.99862658  0.93230152  0.85448633] 

Cross-Predicted Accuracy: 0.980140658389


### Fit and Predict 2016

Fit my model, which cross validation has proved to be fairly succesful, on all of my training data. Then, predict 2016 Year end sales using the Q1 2016 data.

In [30]:
model.fit(X,y)
pred_2016 = model.predict(X_2016)

In [43]:
print "2016 Predicted Total Sales:", sum(pred_2016)
print "2015 Total Sales:",sum(y)

increase = (float(sum(pred_2016) - sum(y))/sum(y)) * 100

print "\n","1 Year Predicted Sales % Increase:", increase

2016 Predicted Total Sales: 290700940.118
2015 Total Sales: 278340275.3

1 Year Predicted Sales % Increase: 4.44084665955
